In [ ]:
import telebot
import pandas as pd

# Replace 'YOUR_BOT_API_TOKEN' with the token you received from the BotFather
API_TOKEN = 'YOUR_BOT_API_TOKEN'

bot = telebot.TeleBot(API_TOKEN)

# Load the CSV file
file_path = 'Tauranga Container Terminal Cargo Schedule.csv'
cargo_schedule_df = pd.read_csv(file_path)

@bot.message_handler(commands=['start'])
def send_welcome(message):
    bot.reply_to(message, "Welcome! How can I assist you today? Use /eta <vessel_name> to get the ETA for a vessel.")

@bot.message_handler(commands=['eta'])
def send_eta(message):
    try:
        vessel_name = ' '.join(message.text.split()[1:]).strip()
        if vessel_name:
            # Find the row corresponding to the vessel name
            vessel_row = cargo_schedule_df[cargo_schedule_df['Vessel'].str.contains(vessel_name, case=False, na=False)]
            if not vessel_row.empty:
                eta = vessel_row.iloc[0]['Current ETA']
                bot.reply_to(message, f"The ETA for {vessel_name} is {eta}.")
            else:
                bot.reply_to(message, f"Vessel {vessel_name} not found.")
        else:
            bot.reply_to(message, "Please provide a vessel name. Usage: /eta <vessel_name>")
    except Exception as e:
        bot.reply_to(message, f"An error occurred: {str(e)}")

@bot.message_handler(func=lambda message: True)
def echo_all(message):
    bot.reply_to(message, message.text)

if __name__ == '__main__':
    bot.polling()